In [8]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine

# Paramètres de connexion
server = 'DESKTOP-UCU1183'
database = 'Staging_Area'

# Connexion pyodbc
conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    "Trusted_Connection=yes;"
)

# Connexion SQLAlchemy (pour pandas / insertions plus tard si besoin)
conn_str = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

# Requête de création de table
query = """
CREATE TABLE [dbo].[Candidature2023_SA] (
    id_et INT,
    date_preinscrits DATE,
    date_convocation DATE,
    date_entretien DATE,
    date_resultat DATE,
    date_enreg DATE,
    date_ne DATE,
    score_final FLOAT,
    moy_bac_et FLOAT,
    resultat VARCHAR(50),
    id_concours INT,
    sexe VARCHAR(20)
);
"""

# Exécuter la création de la table
cursor = conn.cursor()
cursor.execute(query)
conn.commit()   # TRÈS important pour valider les changements
cursor.close()

print("✅ Table 'Candidature2023_SA' créée avec succès dans la base Staging_Area !")


✅ Table 'Candidature2023_SA' créée avec succès dans la base Staging_Area !


In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Paramètres de connexion
server = 'DESKTOP-UCU1183'
database = 'Staging_Area'

# Connexion SQLAlchemy (pratique pour .to_sql)
conn_str = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

# 1. Charger ton CSV corrigé
df = pd.read_csv(r'C:\Users\Acer\Downloads\MOCK_DATA_FINAL_AVEC_SEXE.csv', sep=';')  # ← attention au separateur (';' ou ',' selon ton fichier)

print(f"✅ Données chargées depuis le CSV : {df.shape[0]} lignes, {df.shape[1]} colonnes.")

# 2. Insérer dans SQL Server (dans la table Candidature2023_SA)
df.to_sql(
    'Candidature2023_SA',   # nom de la table dans SQL Server
    con=engine,
    if_exists='append',     # ajoute à la table sans l'écraser
    index=False             # ne pas ajouter une colonne index
)

print("✅ Données insérées avec succès dans la table Candidature2023_SA !")


✅ Données chargées depuis le CSV : 1000 lignes, 12 colonnes.


DataError: (pyodbc.DataError) ('22007', '[22007] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Conversion failed when converting date and/or time from character string. (241) (SQLExecDirectW)')
[SQL: INSERT INTO [Candidature2023_SA] (id_et, date_preinscrits, date_convocation, date_entretien, date_resultat, date_enreg, date_ne, score_final, moy_bac_et, resultat, id_concours, sexe) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?,  ... 6450 characters truncated ...  ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (1, '04/04/2023', '31/07/2023', '02/08/2023', '14/09/2023', '14/09/2023', '23/12/1999', 44.52, 12.09, 'Rejected', 4, 'garcon', 2, '24/06/2023', '16/07/2023', '02/08/2023', '11/09/2023', '14/09/2023', '07/07/2000', 27.6, 17.74, 'Rejected', 4, 'garcon', 3, '30/07/2023', '09/08/2023', '12/09/2023', '14/09/2023', '06/05/2024', '02/04/2003', 46.64, 18.63, 'Rejected', 2, 'fille', 4, '29/07/2023'

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Paramètres de connexion
server = 'DESKTOP-UCU1183'
database = 'Staging_Area'

conn_str = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

# 2. Charger le CSV
df = pd.read_csv(r'C:\Users\Acer\Downloads\MOCK_DATA_FINAL_AVEC_SEXE.csv', sep=',')

print(f"✅ Données chargées depuis le CSV : {df.shape[0]} lignes, {df.shape[1]} colonnes.")

# 3. Conversion des dates en vrai format datetime
date_columns = ['date_preinscrits', 'date_convocation', 'date_entretien', 'date_resultat', 'date_enreg', 'date_ne']

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

# 4. Insertion vers SQL Server
df.to_sql(
    'Candidature2023_SA',
    con=engine,
    if_exists='append',
    index=False
)

print("✅ Données insérées avec succès dans la table Candidature2023_SA !")


ModuleNotFoundError: No module named 'sqlalchemy'

In [3]:
import pandas as pd
# 2. Charger le CSV
df = pd.read_csv(r'C:\Users\Acer\Downloads\MOCK_DATA_FINAL_AVEC_SEXE.csv', sep=',')

print(f"✅ Données chargées depuis le CSV : {df.shape[0]} lignes, {df.shape[1]} colonnes.")

✅ Données chargées depuis le CSV : 1000 lignes, 12 colonnes.


In [4]:
import numpy as np

# Générer des notes réalistes entre 10 et 20
np.random.seed(42)  # pour reproduction des résultats

df['Note_Maths'] = np.random.uniform(10, 20, size=len(df)).round(2)
df['Note_Physique'] = np.random.uniform(8, 20, size=len(df)).round(2)
df['Note_Info'] = np.random.uniform(5, 20, size=len(df)).round(2)
df['Score_Anglais'] = np.random.uniform(50, 100, size=len(df)).round(2)

# Générer Bac type aléatoire
bac_types = ['Sciences', 'Technique', 'Economie', 'Lettres']
df['Bac_Type'] = np.random.choice(bac_types, size=len(df))

# Générer Activités extra-scolaires
activites = ['Club Robotique', 'Hackathon IA', 'Club Sportif', 'Debat', 'Aucune']
df['Activite_Extra'] = np.random.choice(activites, size=len(df))

# Facultatif : détecter "intérêt IA" (0 ou 1 random)
df['Interet_IA'] = np.random.choice([0, 1], size=len(df))


In [5]:
# Vérification rapide
print(df.head())

# Vérifier s'il manque des colonnes critiques
print(df.isnull().sum())


   id_et date_preinscrits date_convocation date_entretien date_resultat  \
0      1       04/04/2023       31/07/2023     02/08/2023    14/09/2023   
1      2       24/06/2023       16/07/2023     02/08/2023    11/09/2023   
2      3       30/07/2023       09/08/2023     12/09/2023    14/09/2023   
3      4       29/07/2023       26/08/2023     31/08/2023    14/09/2023   
4      5       29/07/2023       14/08/2023     17/08/2023    01/09/2023   

   date_enreg     date_ne  score_final  moy_bac_et      resultat  id_concours  \
0  14/09/2023  23/12/1999        44.52       12.09      Rejected            4   
1  14/09/2023  07/07/2000        27.60       17.74      Rejected            4   
2  06/05/2024  02/04/2003        46.64       18.63      Rejected            2   
3  07/01/2024  05/05/2001        57.57        9.69  Waiting List            3   
4  14/09/2023  13/02/2001        33.85        6.46      Rejected            3   

     sexe  Note_Maths  Note_Physique  Note_Info  Score_Anglais

In [2]:
# Sauvegarder dans un nouveau fichier CSV
df.to_csv('MOCK_DATA_FINAL_COMPLET.csv', index=False)

print("✅ Nouveau fichier 'MOCK_DATA_FINAL_COMPLET.csv' créé avec succès !")


NameError: name 'df' is not defined

In [3]:
df.to_csv(r'C:\Users\Acer\Downloads\MOCK_DATA_FINAL_COMPLET.csv', index=False)



NameError: name 'df' is not defined

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# 1. Charger le fichier enrichi
df = pd.read_csv(r'C:\Users\Acer\Downloads\MOCK_DATA_FINAL_COMPLET.csv')

# 2. Features et prétraitement
features = ['Note_Maths', 'Note_Physique', 'Note_Info', 'Score_Anglais', 'Bac_Type', 'Activite_Extra', 'Interet_IA']

X = df[features]

# 3. Encodage One-Hot pour colonnes catégorielles
categorical_cols = ['Bac_Type', 'Activite_Extra']
X = pd.get_dummies(X, columns=categorical_cols)

# 4. Normaliser les notes
scaler = MinMaxScaler()
X[['Note_Maths', 'Note_Physique', 'Note_Info', 'Score_Anglais']] = scaler.fit_transform(X[['Note_Maths', 'Note_Physique', 'Note_Info', 'Score_Anglais']])

# 5. Générer une cible Y simulée pour l'instant (exemple aléatoire)
filiere_possibles = ['Informatique', 'IA', 'Electromecanique', 'Telecom']
import numpy as np
np.random.seed(42)
Y = np.random.choice(filiere_possibles, size=len(df))

# 6. Séparer Train / Test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("✅ Dataset prêt pour l'entraînement :")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")


✅ Dataset prêt pour l'entraînement :
X_train shape: (800, 14)
y_train shape: (800,)


In [9]:
from sklearn.preprocessing import LabelEncoder

# Créer un encodeur
le = LabelEncoder()

# Adapter sur les données
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Vérifions
print(list(le.classes_))
print(y_train_encoded[:5])


[np.str_('Electromecanique'), np.str_('IA'), np.str_('Informatique'), np.str_('Telecom')]
[3 1 3 1 0]


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Nombre de classes (filières)
num_classes = len(le.classes_)

# Créer le modèle
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # 14 features
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Classification mu
    lti-classes
])

# Compiler le modèle
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Résumé du modèle
model.summary()


NameError: name 'le' is not defined

In [1]:
import sys
print(sys.executable)


C:\Users\Acer\AppData\Local\Programs\Python\Python313\python.exe


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1. Charger le fichier CSV
df = pd.read_csv(r'C:\Users\Acer\Downloads\MOCK_DATA_FINAL_COMPLET.csv')

# 2. Sélection des features
features = ['Note_Maths', 'Note_Physique', 'Note_Info', 'Score_Anglais',
            'Bac_Type', 'Activite_Extra', 'Interet_IA']
X = df[features]

# 3. Encodage One-Hot des colonnes catégorielles
categorical_cols = ['Bac_Type', 'Activite_Extra']
X = pd.get_dummies(X, columns=categorical_cols)

# 4. Normalisation des colonnes numériques
scaler = MinMaxScaler()
X[['Note_Maths', 'Note_Physique', 'Note_Info', 'Score_Anglais']] = scaler.fit_transform(
    X[['Note_Maths', 'Note_Physique', 'Note_Info', 'Score_Anglais']]
)

# 5. Simuler la colonne cible Y si elle n'existe pas
filiere_possibles = ['Informatique', 'IA', 'Electromecanique', 'Telecom']
np.random.seed(42)
def assign_filiere(row):
    if row['Note_Info'] > 15:
        return 'IA'
    elif row['Note_Maths'] > 14:
        return 'Informatique'
    elif row['Note_Physique'] > 14:
        return 'Electromecanique'
    else:
        return 'Telecom'

Y = df.apply(assign_filiere, axis=1)


# 6. Encodage des labels (y)
le = LabelEncoder()
y_encoded = le.fit_transform(Y)
num_classes = len(le.classes_)

# 7. Split Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 8. Créer le modèle
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# 9. Compiler le modèle
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 10. Résumé
model.summary()

# 11. Entraînement
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# 12. Évaluation
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc:.2%}")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 128)                 │           1,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,436 (40.77 KB)

 Trainable params: 10,436 (40.77 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.3559 - loss: 1.3043 - val_accuracy: 0.5125 - val_loss: 1.2495
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5607 - loss: 1.1685 - val_accuracy: 0.5562 - val_loss: 1.1386
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6509 - loss: 1.0270 - val_accuracy: 0.6250 - val_loss: 0.9874
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7255 - loss: 0.8322 - val_accuracy: 0.6625 - val_loss: 0.8233
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7881 - loss: 0.6512 - val_accuracy: 0.7437 - val_loss: 0.6825
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8399 - loss: 0.5369 - val_accuracy: 0.7563 - val_loss: 0.5619
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8760 - loss: 0.4422 - val_accuracy: 0.7688 - val_loss: 0.4986
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8837 - loss: 0.3832 - val_accuracy: 0.8250 - v

In [13]:
model.save("smartadmit_model.keras")


In [10]:
# Exemple de prédiction
new_student = pd.DataFrame([{
    'Note_Maths': 14,
    'Note_Physique': 12,
    'Note_Info': 17,
    'Score_Anglais': 15,
    'Bac_Type': 'Scientifique',
    'Activite_Extra': 'Sport',
    'Interet_IA': 1
}])

# Encodage et normalisation identiques
new_student = pd.get_dummies(new_student)
missing_cols = set(X.columns) - set(new_student.columns)
for col in missing_cols:
    new_student[col] = 0
new_student = new_student[X.columns]  # re-aligner l'ordre des colonnes

# Prédiction
prediction = model.predict(new_student)
predicted_class_index = np.argmax(prediction)
predicted_filiere = le.classes_[predicted_class_index]
print(f"🎯 Filière recommandée : {predicted_filiere}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
🎯 Filière recommandée : IA
